In [1]:
# default_exp rank

In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
# export
import os
import sys
sys.path.append('../mylib/')
from code.val import main as val
from code.eda import * 
from code.config import args
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import math  
from sklearn.metrics.pairwise import cosine_similarity
from loguru import logger
from utils.pickle import PickleWrapper

In [5]:
# export
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

# val

In [6]:
# export
def val1(df, topk, click_val):
    """
    """
    sr = df.groupby('user_id')['item_id_pred'].agg(list)
    r_list = []
    pred_num = click_val.shape[0]
    for line in click_val.itertuples():
        try:
#             t = [i[0] for i in sr[line.user_id]]
            df.loc[df.user_id==line.user_id, 'item_id_true'] = line.item_id
            t = sr[line.user_id]
#             print(t)
            i = t.index(line.item_id)
        except:
            i = 99999

        r_list.append(i)
    r_series = pd.Series(r_list)
    for i in range(50, topk+1, 50):
        logger.info(f'recall{i}:{r_series[r_series<i].shape[0]/pred_num}')
        
#     r_series[r_series<9999].plot(kind='box', title='pp', rot=45)
    return df

# load click data

In [8]:
# export
all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)

2020-05-30 18:18:08.389 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 0
2020-05-30 18:18:09.550 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-05-30 18:18:09.555 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 1
2020-05-30 18:18:10.636 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-05-30 18:18:10.637 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 2
2020-05-30 18:18:11.697 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-05-30 18:18:11.699 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 3
2020-05-30 18:18:12.811 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-05-30 18:18:12.815 | INFO     | code.ed

In [6]:
# export
item_feat = get_item_data()

/Users/luoyonggui/PycharmProjects/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [208]:
item_feat.iloc[:, 1:129].shape

(108916, 128)

In [209]:
item_feat.iloc[:, 129:257].shape

(108916, 128)

In [7]:
# export
item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

item_feat.set_index('item_id', inplace=True)

In [45]:
# export
# phase = [0, 1, 2, 3, 5, 6]

phase = [4]
# match_num = 200
match_num = 500

In [185]:
# 计算每个phase的item_id user_cnt  # 貌似每个phase的item只会被user唯一点击一次，那就没必要统计了。。。
def t(s):
    return s.nunique()
all_click_df[0][0].groupby('item_id')['user_id'].agg(t)

item_id
1          4
1000      13
100000     8
10001      5
100013     5
          ..
99990      6
99991      3
99993      5
99994      5
99998      5
Name: user_id, Length: 40775, dtype: int64

In [177]:
# temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
temp_ = all_click_df[0][0].groupby('item_id')['user_id'].agg(list)

In [179]:
temp_

item_id
1                               [1489, 10088, 13420, 19312]
1000      [14546, 9531, 32114, 6934, 20558, 30360, 33492...
100000    [20800, 17064, 33901, 5699, 8541, 1940, 20705,...
10001                      [6572, 17178, 25162, 6383, 8264]
100013                    [18924, 32480, 7696, 6319, 32706]
                                ...                        
99990              [30399, 8084, 2969, 21225, 32152, 34183]
99991                                  [9430, 32852, 31855]
99993                   [23622, 22023, 12546, 31661, 21784]
99994                     [9287, 6305, 28964, 17648, 22420]
99998                    [24217, 9796, 20948, 25551, 16599]
Name: user_id, Length: 40775, dtype: object

In [178]:
temp_.str.len().describe()

count    40775.000000
mean         6.409246
std          6.358793
min          1.000000
25%          4.000000
50%          5.000000
75%          7.000000
max        234.000000
Name: user_id, dtype: float64

In [ ]:
for p in phase:
    # 获取test用户的近n次点击
    temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
    temp_ = pd.DataFrame(temp_)
    for i in range(5):
        temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        
    # 计算每个phase的item_id cnt
    all_click_df[0][0].groupby('item_id')['user_id'].count()

# load match result

In [43]:
# export
def load_match_items():
#     r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527.pkl'))

#     r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527.pkl'))
    
    r_itemcf_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527_phase4.pkl'))

    r_binn_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527_phase4.pkl'))

    
    r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_0527.pkl'))

#     r_itemcf = r_itemcf + r_itemcf_phase4
#     r_binn = r_binn + r_binn_phase4
#     r_itemcf_yl = [r_itemcf_yl[0], r_itemcf_yl[1],r_itemcf_yl[2],r_itemcf_yl[3],r_itemcf_yl[5],r_itemcf_yl[6],r_itemcf_yl[4]]
#     return r_itemcf, r_binn, r_itemcf_yl
    return r_itemcf_phase4, r_binn_phase4, [r_itemcf_yl[4]]


In [44]:
r_itemcf, r_binn, r_itemcf_yl = load_match_items()

In [46]:
# export

def data_preporcess(recall_list, match_num, phase, mode='train'):
    
    itemcf, r_binn, r_itemcf_yl = recall_list
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    for i in range(len(phase)):
        r_itemcf[i]['phase'] = phase[i]
        r_binn[i]['phase'] = phase[i]
        r_itemcf_yl[i]['phase'] = phase[i]
        df1 = df1.append(r_itemcf[i])
        df2 = df2.append(r_binn[i])
        df3 = df3.append(r_itemcf_yl[i])

    df1_ = df1[df1['rank']<(match_num+1)]
    df2_ = df2[df2['rank']<(match_num+1)]
    df3_ = df3[df3['rank']<(match_num+1)]

    print('merge Multi-channel recall...')
    cols = 'user_id item_id_pred phase'.split()
    
    if mode == 'train':
        cols = 'user_id item_id_pred item_id_true phase'.split()
    df = pd.merge(pd.merge(df1_, df2_, on=cols, how='outer'), 
                  df3_, on=cols, how='outer')

    df = df.fillna(0)
    if mode == 'train':
        df['label'] = (df['item_id_pred'] == df['item_id_true']).map(int)


    dft = pd.DataFrame()
    for p in tqdm(phase):
        # 获取test用户的近n次点击
        temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
        temp_ = pd.DataFrame(temp_)
        for i in range(0, 5):
            temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        dft = dft.append(temp_.reset_index())
        # 计算每个phase的item_id cnt
        t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
        df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

    df = pd.merge(df, dft.drop(columns='item_id'))

    df['item_id_pred_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df['item_id_pred_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

    for c in tqdm('last_1 last_2 last_3'.split()):
        df[f'{c}_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

        df[f'{c}_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))


    # 太大了，不建议存储    
    # df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))

    # df = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))
    return df

In [47]:
df = data_preporcess([r_itemcf, r_binn, r_itemcf_yl], match_num, phase)

merge Multi-channel recall...


100%|██████████| 3/3 [03:43<00:00, 74.41s/it]


# FE

In [48]:
def fe(df):
    for i in tqdm('rank score'.split()):
        df[f'{i}1_sub_{i}2'] = df[f'{i}_x']-df[f'{i}_y']
        df[f'{i}1_add_{i}2'] = df[f'{i}_x']+df[f'{i}_y']
        df[f'{i}1_mul_{i}2'] = df[f'{i}_x']*df[f'{i}_y']

        df[f'{i}1_sub_{i}3'] = df[f'{i}_x']-df[f'{i}']
        df[f'{i}1_add_{i}3'] = df[f'{i}_x']+df[f'{i}']
        df[f'{i}1_mul_{i}3'] = df[f'{i}_x']*df[f'{i}']

        df[f'{i}2_sub_{i}3'] = df[f'{i}_y']-df[f'{i}']
        df[f'{i}2_add_{i}3'] = df[f'{i}_y']+df[f'{i}']
        df[f'{i}2_mul_{i}3'] = df[f'{i}_y']*df[f'{i}']

    df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last_1_text_vec']).map(sum)

    df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last_1_img_vec']).map(sum)

    df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last_2_text_vec']).map(sum)
    df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last_2_img_vec']).map(sum)

    df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last_3_text_vec']).map(sum)
    df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last_3_img_vec']).map(sum)

#     df['sim4_text'] = (df['item_id_pred_text_vec'] * df['last_4_text_vec']).map(sum)
#     df['sim4_img'] = (df['item_id_pred_img_vec'] * df['last_4_img_vec']).map(sum)

#     df['sim5_text'] = (df['item_id_pred_text_vec'] * df['last_5_text_vec']).map(sum)
#     df['sim5_img'] = (df['item_id_pred_img_vec'] * df['last_5_img_vec']).map(sum)
    
    

    df['sim1_text_img'] = df['sim1_text'] *  df['sim1_img']
    df['sim2_text_img'] = df['sim2_text'] *  df['sim2_img']
    df['sim3_text_img'] = df['sim3_text'] *  df['sim3_img']
#     df['sim4_text_img'] = df['sim4_text'] *  df['sim4_img']
#     df['sim5_text_img'] = df['sim5_text'] *  df['sim5_img']

    df['sim12_text'] = df['sim1_text'] + df['sim2_text']
    df['sim123_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text']
#     df['sim1234_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text']
#     df['sim12345_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text'] + df['sim5_text']

    df['sim12_img'] = df['sim1_img'] + df['sim2_img']
    df['sim123_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img']
#     df['sim1234_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img']
#     df['sim12345_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img'] + df['sim5_img']

    df['sim12_text_img'] = df['sim1_text_img'] + df['sim2_text_img']
    df['sim123_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img']
#     df['sim1234_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img']
#     df['sim12345_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img'] + df['sim5_text_img']
 
    
    return df

In [49]:
df = fe(df)

100%|██████████| 2/2 [00:00<00:00, 21.75it/s]


In [50]:
df.head()

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label,item_cnt,last_1,last_2,last_3,last_4,last_5,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score2_sub_score3,score2_add_score3,score2_mul_score3,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img
0,15,81372,1.298322,1.0,112749,4,0.421248,5.0,0.110717,1.0,0,5,80495,76453,19981,NaN,NaN,"[1.0933144092559814, 0.10106278955936432, -0.4...","[-1.8858782052993772, -2.5769615173339844, -4....","[1.0933144092559814, 0.10106278955936432, -0.4...","[-1.8858782052993772, -2.5769615173339844, -4....","[1.0933144092559814, 0.10106278955936432, -0.4...","[-1.8858782052993772, -2.5769615173339844, -4....","[1.0933144092559814, 0.10106278955936432, -0.4...","[-1.8858782052993772, -2.5769615173339844, -4....",-4.0,6.0,5.0,0.0,2.0,1.0,4.0,6.0,5.0,0.877074,1.719570,0.546915,1.187604,1.409039,0.143747,0.310530,0.531965,0.046639,768.148443,609.823371,768.148443,609.823371,768.148443,609.823371,468434.872455,468434.872455,468434.872455,1536.296885,2304.445328,1219.646741,1829.470112,9.368697e+05,1.405305e+06
1,15,54598,1.008645,2.0,112749,4,0.643689,1.0,0.095391,2.0,0,32,80495,76453,19981,NaN,NaN,"[0.1712874472141266, 3.521602153778076, -0.832...","[-2.2568836212158203, -2.9105024337768555, -0....","[0.1712874472141266, 3.521602153778076, -0.832...","[-2.2568836212158203, -2.9105024337768555, -0....","[0.1712874472141266, 3.521602153778076, -0.832...","[-2.2568836212158203, -2.9105024337768555, -0....","[0.1712874472141266, 3.521602153778076, -0.832...","[-2.2568836212158203, -2.9105024337768555, -0....",1.0,3.0,2.0,0.0,4.0,4.0,-1.0,3.0,2.0,0.364956,1.652334,0.649253,0.913254,1.104036,0.096216,0.548298,0.739080,0.061402,947.282869,556.999719,947.282869,556.999719,947.282869,556.999719,527636.292182,527636.292182,527636.292182,1894.565738,2841.848607,1113.999439,1670.999158,1.055273e+06,1.582909e+06
2,15,10443,0.885485,3.0,112749,4,0.524099,2.0,0.053269,12.0,0,8,80495,76453,19981,NaN,NaN,"[1.4121811389923096, -2.1995816230773926, -2.8...","[-3.017137289047241, 1.343820333480835, -0.925...","[1.4121811389923096, -2.1995816230773926, -2.8...","[-3.017137289047241, 1.343820333480835, -0.925...","[1.4121811389923096, -2.1995816230773926, -2.8...","[-3.017137289047241, 1.343820333480835, -0.925...","[1.4121811389923096, -2.1995816230773926, -2.8...","[-3.017137289047241, 1.343820333480835, -0.925...",1.0,5.0,6.0,-9.0,15.0,36.0,-10.0,14.0,24.0,0.361385,1.409584,0.464082,0.832216,0.938753,0.047169,0.470830,0.577368,0.027918,1159.044501,609.852245,1159.044501,609.852245,1159.044501,609.852245,706845.890449,706845.890449,706845.890449,2318.089002,3477.133503,1219.704489,1829.556734,1.413692e+06,2.120538e+06
3,15,27982,0.835691,4.0,112749,4,0.184671,35.0,0.075090,3.0,0,4,80495,76453,19981,NaN,NaN,"[2.076242685317993, 0.8410365581512451, -0.575...","[-2.7045726776123047, -1.5759243965148926, -1....","[2.076242685317993, 0.8410365581512451, -0.575...","[-2.7045726776123047, -1.5759243965148926, -1....","[2.076242685317993, 0.8410365581512451, -0.575...","[-2.7045726776123047, -1.5759243965148926, -1....","[2.076242685317993, 0.8410365581512451, -0.575...","[-2.7045726776123047, -1.5759243965148926, -1....",-31.0,39.0,140.0,1.0,7.0,12.0,32.0,38.0,105.0,0.651020,1.020361,0.154328,0.760601,0.910780,0.062752,0.109581,0.259760,0.013867,887.595643,683.644518,887.595643,683.644518,887.595643,683.644518,606799.895701,606799.895701,606799.895701,1775.191285,2662.786928,1367.289037,2050.933555,1.213600e+06,1.820400e+06
4,15,73953,

In [51]:
df.shape

(1681258, 58)

In [80]:
df1.shape, df2.shape, df3.shape, 

((5186489, 6), (4248843, 6), (4996488, 6))

In [81]:
df1.columns

Index(['user_id', 'item_id_pred', 'score', 'rank', 'item_id_true', 'phase'], dtype='object')

In [141]:
df.shape

(9536065, 10)

In [143]:
df.head()

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,0
2,11,10528,5.246875,3.0,8052,0,0.461508,52.0,0.067450,11.0,0
3,11,113564,5.100132,4.0,8052,0,0.386057,66.0,0.136157,1.0,0
4,11,20389,4.636769,5.0,8052,0,0.415892,62.0,0.048790,19.0,0


In [144]:
df.label.value_counts()

0    9533232
1       2833
Name: label, dtype: int64

In [145]:
df.user_id.nunique()

10373

In [147]:
df[df.label==1]

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label
101,11,8052,1.900682,102.0,8052,0,0.324958,87.0,0.018896,82.0,1
1039,44,25164,2.845496,40.0,25164,0,0.207407,180.0,0.037454,29.0,1
2017,66,43836,22.465015,18.0,43836,0,1.488082,18.0,0.100061,13.0,1
3512,132,117039,15.770346,13.0,117039,0,0.955082,21.0,0.174193,2.0,1
8675,319,112967,4.998704,176.0,112967,0,0.280866,201.0,0.079765,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...
9485044,28771,30238,0.000000,0.0,30238,6,0.000000,0.0,0.055383,13.0,1
9503352,30696,30780,0.000000,0.0,30780,6,0.000000,0.0,0.000044,245.0,1
9514064,28749,34356,0.000000,0.0,34356,6,0.000000,0.0,0.001915,412.0,1
9515694,2591,18934,0.000000,0.0,18934,6,0.000000,0.0,0.024958,49.0,1


In [202]:
df.user_id.nunique()

10373

In [201]:
dft.user_id.nunique()

10373

In [14]:
dft

,user_id,item_id,last_4,last_5
0,1001,"[76967, 2827, 11599]",NaN,NaN
1,10032,"[109286, 102996, 105116, 55841, 111687]",102996,109286
2,10043,"[28313, 40108, 33417, 37789, 94638, 60146]",33417,40108
3,10054,"[3267, 66757, 99792, 69011, 74906, 45861, 45652]",69011,99792
4,10065,"[24421, 12510, 43671, 81447, 13429, 41118, 411...",13429,81447
...,...,...,...,...
1816,9928,"[35046, 13255, 3111, 1386, 14552, 6950, 110758...",89727,117577
1817,9939,"[59450, 98514, 101033, 67247, 84198, 27172, 94...",67247,101033
1818,996,"[105489, 112335, 104526, 112448, 110288, 11244...",87846,25594
1819,9961,"[54961, 52178, 34923, 95221, 106441, 83545, 87...",98886,87494


In [11]:
df.shape

(9536065, 24)

In [12]:
df.head(2)

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label,item_cnt,item_id,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0,47,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48..."
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,0,5,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13..."


In [53]:
df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features_phase4.pkl'))

In [52]:
temp_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features.pkl'))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/contest/kdd_debiasing/data_gen/dft500_features.pkl'

In [21]:
temp_.columns

Index(['user_id', 'item_id_pred', 'score_x', 'rank_x', 'item_id_true', 'phase',
       'score_y', 'rank_y', 'score', 'rank', 'label', 'item_cnt', 'item_id',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img'],
      dtype='object')

In [19]:
df.head(2)

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label,item_cnt,item_id,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec,last_4,last_5,last_4_text_vec,last_4_img_vec,last_5_text_vec,last_5_img_vec,sim4_text,sim4_img,sim5_text,sim5_img
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0,47,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...",22113,23323,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...",644.601963,461.714453,644.601963,461.714453
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,0,5,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...",22113,23323,"[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...",692.266290,515.405634,692.266290,515.405634


In [22]:
df = pd.merge(temp_, df['user_id item_id_pred sim4_text sim4_img sim5_text sim5_img'.split()])

In [24]:
temp_.shape

(9536065, 22)

In [23]:
df.shape

(9536641, 26)

In [25]:
df.drop_duplicates('user_id item_id_pred'.split(), inplace=True)

In [26]:
df.shape

(9535777, 26)

In [30]:
df.columns

Index(['user_id', 'item_id_pred', 'score_x', 'rank_x', 'item_id_true', 'phase',
       'score_y', 'rank_y', 'score', 'rank', 'label', 'item_cnt', 'item_id',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text', 'sim4_img',
       'sim5_text', 'sim5_img'],
      dtype='object')

# model

In [26]:
#export
def get_model_input(df, mode='train'):
    from catboost import Pool, CatBoostClassifier

    cols = ['score_x', 'rank_x', 
       'score_y', 'rank_y', 'score', 'rank', 'item_cnt',
       'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text', 'sim4_img',
       'sim5_text', 'sim5_img', 'rank1_sub_rank2', 'rank1_add_rank2',
       'rank1_mul_rank2', 'rank1_sub_rank3', 'rank1_add_rank3',
       'rank1_mul_rank3', 'rank2_sub_rank3', 'rank2_add_rank3',
       'rank2_mul_rank3', 'score1_sub_score2', 'score1_add_score2',
       'score1_mul_score2', 'score1_sub_score3', 'score1_add_score3',
       'score1_mul_score3', 'score2_sub_score3', 'score2_add_score3',
       'score2_mul_score3', 'sim1_text_img', 'sim2_text_img', 'sim3_text_img',
       'sim4_text_img', 'sim5_text_img', 'sim12_text', 'sim123_text',
       'sim1234_text', 'sim12345_text', 'sim12_img', 'sim123_img',
       'sim1234_img', 'sim12345_img', 'sim12_text_img', 'sim123_text_img',
       'sim1234_text_img', 'sim12345_text_img']
    df['fscore'] = 0
    df['frank'] = 999
    cat_cols = []
    if mode=='train':
        X_train, X_test, y_train, y_test = df.loc[df['phase']<6, cols], df.loc[df['phase']==6, cols], df.loc[df['phase']<6, 'label'],\
                                               df.loc[df['phase']==6, 'label']                 

        
        train_data = Pool(data=X_train,
                           label=y_train,
                           cat_features=cat_cols)
        val_data = Pool(data=X_test, label=y_test, cat_features=cat_cols) 
        return train_data, val_data, cat_cols
    X_test = df[cols]
    test_data = Pool(data=X_test, cat_features=cat_cols) 
    return test_data, cat_cols

In [127]:
# df.loc[:, 'rank_x'] = df.rank_x.map(lambda x: 999 if x==0 else x)
# df.loc[:, 'rank_y'] = df.rank_x.map(lambda x: 999 if x==0 else x)
# df.loc[:, 'rank'] = df.rank_x.map(lambda x: 999 if x==0 else x)

In [33]:
df.columns

Index(['user_id', 'item_id_pred', 'score_x', 'rank_x', 'item_id_true', 'phase',
       'score_y', 'rank_y', 'score', 'rank', 'label', 'item_cnt', 'item_id',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text', 'sim4_img',
       'sim5_text', 'sim5_img', 'rank1_sub_rank2', 'rank1_add_rank2',
       'rank1_mul_rank2', 'rank1_sub_rank3', 'rank1_add_rank3',
       'rank1_mul_rank3', 'rank2_sub_rank3', 'rank2_add_rank3',
       'rank2_mul_rank3', 'score1_sub_score2', 'score1_add_score2',
       'score1_mul_score2', 'score1_sub_score3', 'score1_add_score3',
       'score1_mul_score3', 'score2_sub_score3', 'score2_add_score3',
       'score2_mul_score3', 'sim1_text_img', 'sim2_text_img', 'sim3_text_img',
       'sim4_text_img', 'sim5_text_img', 'sim12_text', 'sim123_text',
       'sim1234_text', 'sim12345_text', 'sim12_img', 'sim123_img',
       'sim1234_img', 'sim12345_img', 'sim12_text_img', 'sim123_text_img',
  

In [35]:
df['phase'].value_counts()

6    1820867
5    1776430
3    1599358
2    1565472
1    1524428
0    1249222
Name: phase, dtype: int64

In [39]:
df.loc[df['phase']==6].user_id.nunique()

1821

In [40]:
df.loc[df['phase']==6].shape

(1820867, 63)

In [ ]:
# export
def get_model():
    from catboost import Pool, CatBoostClassifier
    params = {
        'iterations': 360,
        'learning_rate': 0.1,
        'random_seed': 144,
        'custom_metric': 'F1',
        'loss_function': 'Logloss',
        'class_weights': [1, 20],
    #     'use_best_model': False
        }
    print(params)
    model = CatBoostClassifier(**params)
    return model

In [108]:
# val1(r_itemcf[-1], 100, all_click_df[6][1])
# 2020-05-27 17:39:59.717 | INFO     | __main__:val1:24 - recall50:0.15595826468973092
# 2020-05-27 17:39:59.728 | INFO     | __main__:val1:24 - recall100:0.18231740801757276


# fit

In [46]:
r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

2020-05-28 17:35:07.937 | INFO     | __main__:val1:24 - recall50:0.17298187808896212
2020-05-28 17:35:07.940 | INFO     | __main__:val1:24 - recall100:0.21087314662273476


In [42]:
r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
2020-05-28 16:54:12.975 | INFO     | __main__:val1:24 - recall50:0.16035145524437122
2020-05-28 16:54:12.994 | INFO     | __main__:val1:24 - recall100:0.20043931905546403


In [47]:
# model.save_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500_feat_last12345.dump'))

In [43]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

0


2020-05-28 16:55:58.189 | INFO     | __main__:val1:24 - recall50:0.16474464579901152
2020-05-28 16:55:58.191 | INFO     | __main__:val1:24 - recall100:0.20922570016474465


1


2020-05-28 16:57:14.376 | INFO     | __main__:val1:24 - recall50:0.1658429434376716
2020-05-28 16:57:14.378 | INFO     | __main__:val1:24 - recall100:0.20922570016474465


2


...

2020-05-28 17:04:27.437 | INFO     | __main__:val1:24 - recall50:0.1696869851729819
2020-05-28 17:04:27.438 | INFO     | __main__:val1:24 - recall100:0.2070291048874245


8


2020-05-28 17:05:36.891 | INFO     | __main__:val1:24 - recall50:0.1702361339923119
2020-05-28 17:05:36.892 | INFO     | __main__:val1:24 - recall100:0.20812740252608458


9


2020-05-28 17:06:48.050 | INFO     | __main__:val1:24 - recall50:0.171334431630972
2020-05-28 17:06:48.052 | INFO     | __main__:val1:24 - recall100:0.2086765513454146


In [44]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

0


2020-05-28 17:10:27.420 | INFO     | __main__:val1:24 - recall50:0.171334431630972
2020-05-28 17:10:27.424 | INFO     | __main__:val1:24 - recall100:0.2086765513454146


1


2020-05-28 17:11:40.422 | INFO     | __main__:val1:24 - recall50:0.17078528281164196
2020-05-28 17:11:40.424 | INFO     | __main__:val1:24 - recall100:0.2086765513454146


2


...

2020-05-28 17:18:51.216 | INFO     | __main__:val1:24 - recall50:0.17243272926963207
2020-05-28 17:18:51.217 | INFO     | __main__:val1:24 - recall100:0.21032399780340472


8


2020-05-28 17:20:03.536 | INFO     | __main__:val1:24 - recall50:0.171334431630972
2020-05-28 17:20:03.540 | INFO     | __main__:val1:24 - recall100:0.20812740252608458


9


2020-05-28 17:21:18.146 | INFO     | __main__:val1:24 - recall50:0.17078528281164196
2020-05-28 17:21:18.148 | INFO     | __main__:val1:24 - recall100:0.20922570016474465


In [48]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

0


2020-05-28 11:55:56.561 | INFO     | __main__:val1:24 - recall50:0.1702361339923119
2020-05-28 11:55:56.564 | INFO     | __main__:val1:24 - recall100:0.2114222954420648


1


2020-05-28 11:57:09.415 | INFO     | __main__:val1:24 - recall50:0.1696869851729819
2020-05-28 11:57:09.416 | INFO     | __main__:val1:24 - recall100:0.2097748489840747


2


...

2020-05-28 12:00:40.302 | INFO     | __main__:val1:24 - recall50:0.1685886875343218
2020-05-28 12:00:40.305 | INFO     | __main__:val1:24 - recall100:0.20593080724876442


5


2020-05-28 12:01:51.198 | INFO     | __main__:val1:24 - recall50:0.16913783635365184
2020-05-28 12:01:51.199 | INFO     | __main__:val1:24 - recall100:0.20483250961010435


6


KeyboardInterrupt: 

In [36]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

0


2020-05-28 11:27:10.489 | INFO     | __main__:val1:24 - recall50:0.1685886875343218
2020-05-28 11:27:10.492 | INFO     | __main__:val1:24 - recall100:0.2086765513454146


1


2020-05-28 11:28:16.959 | INFO     | __main__:val1:24 - recall50:0.16749038989566173
2020-05-28 11:28:16.962 | INFO     | __main__:val1:24 - recall100:0.20593080724876442


2


...

2020-05-28 11:35:16.526 | INFO     | __main__:val1:24 - recall50:0.16694124107633168
2020-05-28 11:35:16.528 | INFO     | __main__:val1:24 - recall100:0.20538165842943437


8


2020-05-28 11:36:25.212 | INFO     | __main__:val1:24 - recall50:0.16694124107633168
2020-05-28 11:36:25.214 | INFO     | __main__:val1:24 - recall100:0.20483250961010435


9


KeyboardInterrupt: 

# predict

In [9]:
item_feat = get_item_data()

item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

item_feat.set_index('item_id', inplace=True)

/root/data1/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [18]:
r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_test_0527.pkl'))

r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_test_0527.pkl'))

r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_test_0527.pkl'))


In [19]:
phase = [0, 1, 2, 3, 4, 5, 6]
match_num = 500

df = data_preporcess([r_itemcf, r_binn, r_itemcf_yl], match_num, phase, mode='predict')


merge Multi-channel recall...


100%|██████████| 5/5 [40:43<00:00, 488.77s/it]


In [22]:
df = fe(df)


In [24]:
cols = ['user_id', 'item_id_pred', 'phase', 'score_x', 'rank_x', 
       'score_y', 'rank_y', 'score', 'rank', 'item_cnt',
       'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text', 'sim4_img',
       'sim5_text', 'sim5_img', 'rank1_sub_rank2', 'rank1_add_rank2',
       'rank1_mul_rank2', 'rank1_sub_rank3', 'rank1_add_rank3',
       'rank1_mul_rank3', 'rank2_sub_rank3', 'rank2_add_rank3',
       'rank2_mul_rank3', 'score1_sub_score2', 'score1_add_score2',
       'score1_mul_score2', 'score1_sub_score3', 'score1_add_score3',
       'score1_mul_score3', 'score2_sub_score3', 'score2_add_score3',
       'score2_mul_score3', 'sim1_text_img', 'sim2_text_img', 'sim3_text_img',
       'sim4_text_img', 'sim5_text_img', 'sim12_text', 'sim123_text',
       'sim1234_text', 'sim12345_text', 'sim12_img', 'sim123_img',
       'sim1234_img', 'sim12345_img', 'sim12_text_img', 'sim123_text_img',
       'sim1234_text_img', 'sim12345_text_img']
df[cols].to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features.pkl'))

In [27]:
test_data, cat_cols = get_model_input(df, mode='predict')


In [19]:
# set(df.user_id)-set(dft.user_id) # {'14997'}

{'14997'}

In [28]:
from catboost import Pool, CatBoostClassifier

In [29]:
model = CatBoostClassifier()
model.load_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500_feat_last12345.dump'))

In [31]:
df.loc[:, 'fscore'] = model.predict_proba(test_data)[:, 1]

In [32]:
df.loc[:, 'frank'] = df.groupby('user_id')['fscore'].rank(method='first', ascending=False)

In [33]:
df.head()

,user_id,item_id_pred,score_x,rank_x,phase,score_y,rank_y,score,rank,item_cnt,last_1,last_2,last_3,last_4,last_5,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec,last_4_text_vec,last_4_img_vec,last_5_text_vec,last_5_img_vec,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score2_sub_score3,score2_add_score3,score2_mul_score3,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,sim4_text,sim4_img,sim5_text,sim5_img,sim1_text_img,sim2_text_img,sim3_text_img,sim4_text_img,sim5_text_img,sim12_text,sim123_text,sim1234_text,sim12345_text,sim12_img,sim123_img,sim1234_img,sim12345_img,sim12_text_img,sim123_text_img,sim1234_text_img,sim12345_text_img,fscore,frank
0,11,10528,9.343545,1.0,0,0.749194,21.0,0.110274,2.0,7.0,28210,64689,20317,22113,23323,"[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...",-20.0,22.0,21.0,-1.0,3.0,2.0,19.0,23.0,42.0,8.594351,10.092739,7.000128,9.233271,9.453819,1.030350,0.638920,0.859468,0.082617,927.854410,776.236688,927.854410,776.236688,927.854410,776.236688,927.854410,776.236688,927.854410,776.236688,720234.634356,720234.634356,720234.634356,720234.634356,720234.634356,1855.708819,2783.563229,3711.417639,4639.272049,1.552473e+03,2.328710e+03,3.104947e+03,3.881183e+03,1.440469e+06,2.160704e+06,2.880939e+06,3.601173e+06,0.229424,1.0
1,11,79868,6.949118,2.0,0,2.023002,1.0,0.007360,322.0,47.0,28210,64689,20317,22113,23323,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...",1.0,3.0,2.0,-320.0,324.0,644.0,-321.0,323.0,322.0,4.926116,8.972120,14.058081,6.941758,6.956478,0.051147,2.015642,2.030362,0.014890,644.601963,461.714453,644.601963,461.714453,644.601963,461.714453,644.601963,461.714453,644.601963,461.714453,297622.042561,297622.042561,297622.042561,297622.042561,297622.042561,1289.203926,1933.805889,2578.407852,3223.009814,9.234289e+02,1.385143e+03,1.846858e+03,2.308572e+03,5.952441e+05,8.928661e+05,1.190488e+06,1.488110e+06,0.013338,70.0
2,11,21517,5.841872,3.0,0,0.411440,64.0,0.103648,3.0,9.0,28210,64689,20317,22113,23323,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 

In [34]:
df1 = df.sort_values(['user_id', 'fscore'], ascending=False)

In [35]:
sub = df1[df1.frank<51]

In [36]:
sub = sub.groupby('user_id')['item_id_pred'].agg(list)

In [37]:
sub

user_id
1        [32360, 93601, 103887, 87837, 30677, 46297, 10...
100      [100089, 57797, 112676, 27324, 108507, 113759,...
10001    [27822, 45632, 5173, 25257, 79132, 43575, 6954...
10002    [100447, 68539, 92204, 80136, 74285, 87051, 79...
10003    [91369, 104173, 90074, 39956, 94839, 76521, 54...
                               ...                        
9989     [111222, 107329, 107960, 101475, 74774, 80679,...
9991     [20258, 35820, 63342, 19181, 63380, 7971, 7287...
9992     [30935, 42102, 46191, 10050, 11182, 101330, 64...
9993     [13354, 38752, 80129, 4204, 18861, 10294, 3567...
9994     [46996, 67789, 71249, 14347, 43622, 6973, 8068...
Name: item_id_pred, Length: 12080, dtype: object

In [36]:
sub

user_id
1        [32360, 103887, 93601, 87837, 91290, 46297, 10...
100      [100089, 27324, 112676, 57797, 17927, 30600, 4...
10001    [27822, 45632, 5173, 79132, 69545, 48990, 1098...
10002    [92204, 100447, 80136, 68539, 5525, 74285, 870...
10003    [91369, 31577, 90074, 10153, 104173, 76521, 39...
                               ...                        
9989     [111222, 107329, 101475, 107960, 107968, 74774...
9991     [7971, 20258, 72872, 19181, 63342, 46897, 4856...
9992     [42102, 30935, 46191, 18277, 39725, 101330, 10...
9993     [13354, 16139, 62601, 14061, 51985, 80129, 671...
9994     [67789, 46996, 39354, 6818, 14347, 8068, 71249...
Name: item_id_pred, Length: 12080, dtype: object

In [38]:
sub1 = pd.DataFrame()

for i in range(50):
    sub1[f'c{i}'] = sub.str.get(i)

sub1.applymap(int).to_csv('baseline1_itemcf26666.csv', header=False)

In [39]:
sub1.shape

(12080, 50)